In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
from suntime import Sun
import pytz
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
pd.set_option('display.float_format', lambda x: '%.8f' % x)

# Helper Functions

In [2]:
def datetime_index(dataframe):
    dataframe.columns = dataframe.columns.str.replace(' ', '')
    dataframe['Datetime'] = pd.to_datetime(dataframe['DateTime'])
    dataframe = dataframe.set_index(['Datetime'])
    del dataframe['DateTime']
    return dataframe

def fill_na(dataframe):
    dataframe['WWCode'] = dataframe['WWCode'].fillna(0).astype(int)
    return dataframe

def day_night(dataframe):

    coordinates = [40.239, 33.029] # Ankara
    dataframe['datetime_col'] = dataframe.index
    
    dataframe["Date"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.date.apply(str)
    dataframe["Hour"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.hour
    dataframe["Day_of_Week"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.dayofweek
    dataframe["Quarter"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.quarter
    dataframe["Month"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.month
    dataframe["Year"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.year
    dataframe["Day_of_Year"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.dayofyear
    dataframe["Week"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.week
    dataframe["Week_of_Year"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.weekofyear
    
    sun = Sun(coordinates[0], coordinates[1])
    tz =pytz.timezone('Europe/Istanbul')
    dataframe["Day"]= dataframe[["Date","Hour"]].apply(lambda x : sun.get_local_sunrise_time(pd.to_datetime(x["Date"]).
                                                                date(),local_time_zone=tz).
                                         hour <= x["Hour"] <= sun.get_local_sunset_time(pd.to_datetime(x["Date"]).
                                                                date(),local_time_zone=tz).hour ,axis=1)
    dataframe = dataframe.drop(['Date', 'datetime_col'], axis = 1)
    return dataframe

def wwcode_encoder(dataframe):
    for index, row in dataframe.iterrows():
        if row['WWCode'] < 10:
            dataframe.loc[index, 'WWCode'] = 1
        elif row['WWCode'] < 20:
            dataframe.loc[index, 'WWCode'] = 2
        elif row['WWCode'] < 30:
            dataframe.loc[index, 'WWCode'] = 3
        elif row['WWCode'] < 40:
            dataframe.loc[index, 'WWCode'] = 4
        elif row['WWCode'] < 50:
            dataframe.loc[index, 'WWCode'] = 5
        elif row['WWCode'] < 60:
            dataframe.loc[index, 'WWCode'] = 6
        elif row['WWCode'] < 70:
            dataframe.loc[index, 'WWCode'] = 7
        elif row['WWCode'] < 80:
            dataframe.loc[index, 'WWCode'] = 8
        elif row['WWCode'] < 90:
            dataframe.loc[index, 'WWCode'] = 9
        elif row['WWCode'] < 100:
            dataframe.loc[index, 'WWCode'] = 10
    return dataframe
    
    
def season_encoder(dataframe):
    dataframe['Season'] = 0
    for index, row in dataframe.iterrows():
        if row['Month'] < 3 or row['Month'] == 12:
            dataframe.loc[index, 'Season'] = 1
        elif row['Month'] < 6:
            dataframe.loc[index, 'Season'] = 2
        elif row['Month'] < 9:
            dataframe.loc[index, 'Season'] = 3
        elif row['Month'] < 12:
            dataframe.loc[index, 'Season'] = 4
    return dataframe

def fig_plot():
    plt.figure(figsize = (10,15))
    plt.scatter(Y_test, pred_val)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    
    
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)
    early_stopping_rounds = trial.suggest_int("early_stopping_rounds", 100, 500)
    n_estimators = trial.suggest_int("n_estimators", 0, 10000)
    
    model = XGBRegressor(
        random_state=50,
        tree_method="gpu_hist",
        gpu_id=0,
        predictor="gpu_predictor",
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(
        X_train,
        Y_train,
        early_stopping_rounds=early_stopping_rounds,
        eval_set=[(X_test, Y_test)],
        verbose=1200,
    )
    preds_valid = model.predict(X_test)
    rmse = mean_squared_error(Y_test, preds_valid, squared=False)

    return rmse

# Data Extraction

In [3]:
submission = pd.read_csv('sample_submission.csv')
label = datetime_index(pd.read_csv('generation.csv', delimiter=';', decimal=','))
features = datetime_index(pd.read_csv('temperature.csv', delimiter=';', decimal=','))

In [4]:
x_train  = fill_na(features.loc['2019-01-01':'2021-11-30'])
x_test = day_night(fill_na(features.loc['2021-12-01':'2021-12-31']))
y_train  = label.loc['2019-01-01':'2021-11-30']

C:\Users\asus\AppData\Local\Temp/ipykernel_11304/2852319294.py:24: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataframe["Week"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.week
C:\Users\asus\AppData\Local\Temp/ipykernel_11304/2852319294.py:25: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataframe["Week_of_Year"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.weekofyear


In [5]:
df_train = day_night(pd.concat([ x_train, y_train], axis=1))

C:\Users\asus\AppData\Local\Temp/ipykernel_11304/2852319294.py:24: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataframe["Week"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.week
C:\Users\asus\AppData\Local\Temp/ipykernel_11304/2852319294.py:25: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataframe["Week_of_Year"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.weekofyear


### Preprocessing

In [6]:
x_test.loc[x_test['WWCode'] == 84, 'WWCode'] = 83 

# x_test = wwcode_encoder(x_test)
# df_train = wwcode_encoder(df_train)

In [7]:
df_train = season_encoder(df_train)
x_test = season_encoder(x_test)

In [8]:
# df_train.drop(['ComfortTemperature'], axis=1, inplace=True)
# x_test.drop(['ComfortTemperature'], axis=1, inplace=True)

In [9]:
# df_train

In [10]:
# df_train = pd.get_dummies(df_train, columns = ['WWCode'])
# x_test = pd.get_dummies(x_test, columns = ['WWCode'])

##### Dataset no:1

In [11]:
X_train = df_train.loc['2019-01-01':'2021-10-31'].drop(['Generation'], axis=1)
X_test = df_train.loc['2021-10-01':'2021-11-30'].drop(['Generation'], axis=1)
Y_train = df_train.Generation.loc['2019-01-01':'2021-10-31']
Y_test = df_train.Generation.loc['2021-10-01':'2021-11-30']

##### Dataset no:2

In [12]:
# X_train = df_train.loc['2019-01-01':'2021-04-30'].drop(['Generation'], axis=1)
# X_test = df_train.loc['2021-05-01':'2021-11-30'].drop(['Generation'], axis=1)
# Y_train = df_train.Generation.loc['2019-01-01':'2021-04-30']
# Y_test = df_train.Generation.loc['2021-05-01':'2021-11-30']

In [13]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

[I 2022-02-28 17:11:48,017] A new study created in memory with name: no-name-c95283dd-8491-4da8-b02a-419b50c213d1


[0]	validation_0-rmse:149.54890
[1200]	validation_0-rmse:56.58453
[2400]	validation_0-rmse:49.53490
[3600]	validation_0-rmse:47.01902
[4800]	validation_0-rmse:45.65705
[6000]	validation_0-rmse:44.87630
[7200]	validation_0-rmse:44.48112
[8224]	validation_0-rmse:44.26056


[I 2022-02-28 17:12:15,373] Trial 0 finished with value: 44.26041875707435 and parameters: {'learning_rate': 0.012153553861605607, 'reg_lambda': 0.01851726342352379, 'reg_alpha': 3.2111289969422966e-07, 'subsample': 0.6620956162915191, 'colsample_bytree': 0.5242324239834101, 'max_depth': 1, 'early_stopping_rounds': 441, 'n_estimators': 8225}. Best is trial 0 with value: 44.26041875707435.


[0]	validation_0-rmse:144.50514
[1200]	validation_0-rmse:47.81280
[2400]	validation_0-rmse:44.63902
[3600]	validation_0-rmse:43.80969
[4800]	validation_0-rmse:43.68132
[5541]	validation_0-rmse:43.65009


[I 2022-02-28 17:12:32,035] Trial 1 finished with value: 43.61076245406743 and parameters: {'learning_rate': 0.11534830870314348, 'reg_lambda': 0.0006267027212034012, 'reg_alpha': 7.839928997830204e-05, 'subsample': 0.4340019115757554, 'colsample_bytree': 0.13731504623581353, 'max_depth': 1, 'early_stopping_rounds': 429, 'n_estimators': 8022}. Best is trial 1 with value: 43.61076245406743.


...

[I 2022-02-28 17:18:50,425] Trial 17 finished with value: 19.901230388783606 and parameters: {'learning_rate': 0.021590649061488602, 'reg_lambda': 1.0293271712237263e-08, 'reg_alpha': 9.18889497548421e-06, 'subsample': 0.371481455230913, 'colsample_bytree': 0.7159724816612857, 'max_depth': 5, 'early_stopping_rounds': 251, 'n_estimators': 4759}. Best is trial 15 with value: 18.03343479684596.


[0]	validation_0-rmse:146.06310
[1200]	validation_0-rmse:17.52010
[1673]	validation_0-rmse:17.46552


[I 2022-02-28 17:19:25,473] Trial 18 finished with value: 17.376142690176568 and parameters: {'learning_rate': 0.024948049056494156, 'reg_lambda': 3.217398864284362e-07, 'reg_alpha': 1.38355325650856e-07, 'subsample': 0.9924204541060586, 'colsample_bytree': 0.8744762925042558, 'max_depth': 7, 'early_stopping_rounds': 317, 'n_estimators': 3501}. Best is trial 18 with value: 17.376142690176568.


[0]	validation_0-rmse:148.46158
[928]	validation_0-rmse:20.44868


[I 2022-02-28 17:19:37,885] Trial 19 finished with value: 20.44868095315732 and parameters: {'learning_rate': 0.010566666497524468, 'reg_lambda': 3.893970293686095e-07, 'reg_alpha': 6.703305000735066e-05, 'subsample': 0.972647090261336, 'colsample_bytree': 0.8573791827157653, 'max_depth': 6, 'early_stopping_rounds': 210, 'n_estimators': 929}. Best is trial 18 with value: 17.376142690176568.


In [14]:
optuna_params = study.best_params
optuna_params

{'learning_rate': 0.024948049056494156,
 'reg_lambda': 3.217398864284362e-07,
 'reg_alpha': 1.38355325650856e-07,
 'subsample': 0.9924204541060586,
 'colsample_bytree': 0.8744762925042558,
 'max_depth': 7,
 'early_stopping_rounds': 317,
 'n_estimators': 3501}

In [15]:
model = XGBRegressor(
    random_state=50,
    tree_method="gpu_hist",
    gpu_id=0,
    predictor="gpu_predictor",
    **optuna_params
)
model.fit(
    X_train,
    Y_train,
    early_stopping_rounds=optuna_params['early_stopping_rounds'],
    eval_set=[(X_test, Y_test)],
    verbose=1000,
)

[17:19:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "early_stopping_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:146.06310
[1000]	validation_0-rmse:17.69972
[1673]	validation_0-rmse:17.46552


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8744762925042558,
             early_stopping_rounds=317, enable_categorical=False, gamma=0,
             gpu_id=0, importance_type=None, interaction_constraints='',
             learning_rate=0.024948049056494156, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=3501, n_jobs=8, num_parallel_tree=1,
             predictor='gpu_predictor', random_state=50,
             reg_alpha=1.38355325650856e-07, reg_lambda=3.217398864284362e-07,
             scale_pos_weight=1, subsample=0.9924204541060586,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [16]:
pred_val_optuna = model.predict(X_test)
pred_val_optuna = pd.DataFrame(pred_val_optuna, index=X_test.index, columns=['pred'])

In [17]:
pred_val_optuna = day_night(pred_val_optuna)
pred_val_optuna.pred[pred_val_optuna.Day == False] = 0
pred_val_optuna = pred_val_optuna.pred
pred_val_optuna

C:\Users\asus\AppData\Local\Temp/ipykernel_11304/2852319294.py:24: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataframe["Week"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.week
C:\Users\asus\AppData\Local\Temp/ipykernel_11304/2852319294.py:25: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataframe["Week_of_Year"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.weekofyear
C:\Users\asus\AppData\Local\Temp/ipykernel_11304/1247003563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_val_optuna.pred[pred_val_optuna.Day == False] = 0


Datetime
2021-10-01 00:00:00   0.00000000
2021-10-01 01:00:00   0.00000000
2021-10-01 02:00:00   0.00000000
2021-10-01 03:00:00   0.00000000
2021-10-01 04:00:00   0.00000000
                         ...    
2021-11-30 19:00:00   0.00000000
2021-11-30 20:00:00   0.00000000
2021-11-30 21:00:00   0.00000000
2021-11-30 22:00:00   0.00000000
2021-11-30 23:00:00   0.00000000
Name: pred, Length: 1464, dtype: float32

In [18]:
rmse_optuna = mean_squared_error(Y_test, pred_val_optuna, squared=False)
rmse_optuna

17.31183852534761